In [1]:
import networkx as nx
import pandas as pd
import numpy as np

In [116]:
edges = pd.read_csv(r'dataset\archive\edges.csv')
hero_net = pd.read_csv(r'dataset\archive\hero-network.csv')
nodes = pd.read_csv(r'dataset\archive\nodes.csv')

In [117]:
# Remove last space if present.
hero_net['hero1'] = hero_net['hero1'].apply(lambda row: row.rstrip().rstrip('/'))
hero_net['hero2'] = hero_net['hero2'].apply(lambda row: row.rstrip().rstrip('/'))

# Cut to max 20 characters hero's names.
edges['hero'] = edges['hero'].apply(lambda row: row[:20])

# Remove row with same hero (self-loop).
hero_net.drop(hero_net[hero_net.hero1 == hero_net.hero2].index, inplace=True)
hero_net = hero_net.reset_index(drop=True)